# web page reader && RAG

## 准备

In [1]:
%%time
%%capture

%pip install llama-index llama-index-readers-web

CPU times: user 77.5 ms, sys: 5.11 ms, total: 82.6 ms
Wall time: 22.7 s


In [ ]:
%%time

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
%%time

from llama_index.core import SummaryIndex
from llama_index.readers.web import SimpleWebPageReader
from IPython.display import Markdown, display
import os

CPU times: user 2.73 s, sys: 612 ms, total: 3.34 s
Wall time: 4.35 s


## 获取文档对象

In [3]:
%%time

def get_documents(url):
    return SimpleWebPageReader(html_to_text=True).load_data(
        [url]
    )

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs


In [5]:
%%time

url="https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml"

docs=get_documents(url)

CPU times: user 73.9 ms, sys: 2.03 ms, total: 75.9 ms
Wall time: 127 ms


## 直接将网页传给 LLM

效果不好。问题在于是整个网页内容，LLM没有找到主要新闻内容。 -- 是因为llm上下文长度太小。

In [107]:
%%time

from openai import OpenAI

client = OpenAI(
    base_url = 'http://192.168.0.72:11434/v1',
    api_key='ollama', # required, but unused
)

CPU times: user 50.4 ms, sys: 0 ns, total: 50.4 ms
Wall time: 49.1 ms


## 加大上下文长度

In [108]:
%%time

stream = client.chat.completions.create(
  model="qwen-14b-4k",
  messages=[
    # {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": "你是谁？"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

我是通义千问，由阿里云开发的AI助手。我被设计用来回答各种问题、提供信息和与用户进行对话。有什么我可以帮助你的吗？CPU times: user 44.3 ms, sys: 6.73 ms, total: 51 ms
Wall time: 48.3 s


In [109]:
%%time

file_path = './content.txt'

# Read the content of the file and store it in a string variable
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

CPU times: user 1.07 ms, sys: 0 ns, total: 1.07 ms
Wall time: 1.06 ms


In [110]:
len(content)

6290

In [ ]:
%%time

stream = client.chat.completions.create(
  model="qwen-14b-4k",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，详细归纳一下，形成要点列表。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

# 乱码了

In [7]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings

llm = OpenAILike(model="qwen:14b", 
                 api_base="http://192.168.0.72:3000/v1", 
                 api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
                 is_chat_model=True
                )
Settings.llm =llm

CPU times: user 171 µs, sys: 33 µs, total: 204 µs
Wall time: 207 µs


In [23]:
%%time
%%capture

%pip install beautifulsoup4
%pip install requests

CPU times: user 14 ms, sys: 11.3 ms, total: 25.4 ms
Wall time: 5.82 s


## 使用 beautifulsoup

In [ ]:
效果也不理想。只是获取了完整的html文本。

如果抽取指定内容，比如正文部分，需要自己解析。

In [24]:
%%time

from bs4 import BeautifulSoup
import requests

url = "https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

CPU times: user 126 ms, sys: 7.34 ms, total: 134 ms
Wall time: 276 ms


In [27]:
soup

<!DOCTYPE html>

<html lang="zh-cmn-Hans">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<link href="../css/public.css?20230516" rel="stylesheet" type="text/css"/>
<link href="../css/main.css?2020123101" rel="stylesheet" type="text/css"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="zh-CN" http-equiv="content-language"/>
<meta content="no-transform" http-equiv="Cache-Control">
<meta content="no-siteapp" http-equiv="Cache-Control">
<meta content="pc" name="applicable-device"/>
<meta content="format=xhtml;url=https://m.guancha.cn/andelie/2024_06_06_737185.shtml" name="mobile-agent"/>
<link href="https://m.guancha.cn/andelie/2024_06_06_737185.shtml" media="only screen and (max-width: 970px)" rel="alternate"/>
<meta content="format=html5;url=https://m.guancha.cn/andelie/2024_06_06_737185.shtml" name="mobile-agent"/>
<meta content="" name="Keywords"/>
<meta content="朝鲜半岛问题，台湾问题都是冷战的历史遗留问题，可以说在亚洲，冷战还没有结束，

In [28]:
title_text = soup.find('title').get_text()
title_text

'安德烈·科尔图诺夫：无论经济发展还是和平建设，亚洲都可以给欧洲上一课'

In [30]:
%%time

content=soup.find('div').get_text()

CPU times: user 492 µs, sys: 0 ns, total: 492 µs
Wall time: 495 µs


In [32]:
content[:1000]

'\n\n\n\n\n\n\n\n首页\n\n\n风闻\n\n\n财经\n\n\n国际\n\n\n军事\n\n\n公司\n\n\n科技\n\n\n汽车\n\n\n视频\n\n\n\n\n\n观察员\n\n\n\n\n\n\nAPP下载\n\n\n\n扫一扫\n下载观察者APP\n\n\n \n\n\n\n\n安德烈·科尔图诺夫：无论经济发展还是和平建设，亚洲都可以给欧洲上一课\n\n\n\n\n安德烈·科尔图诺夫俄罗斯国际事务委员会学术委员会主任\n\n\n\n\n\n分享到：\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n2024-06-06 08:12:41\n字号：A-\nA\nA+\n来源：观察者网\n\n\n最后更新: 2024-06-06 13:23:14\n\n\r\n\t【文/观察者网专栏作者 安德烈·科尔图诺夫，翻译/马力】\r\n\n\r\n\t上世纪90年代初，我第一次来到了哈尔滨，这座掩映在丁香树林中的城市给我留下的印象是，这座城市正致力于消除俄罗斯残余痕迹，这是欧洲殖民主义最后一点影响的象征。\r\n\n\r\n\t事实上，这座始建于1898年的城市最初是由中东铁路线上的一个车站逐渐发展而来的，在其建成后的头20年里，哈尔滨具有特殊的治外法权地位，而且实际上已经脱离了首都北京的管辖。长期以来，当地中国人对这段国家主权受到损害的历史有着痛苦的回忆，而且对这段历史非常敏感，对此我们俄罗斯人必须接受并给予理解。\r\n\n\r\n\t然而，在过去的30多年里，人们的态度似乎发生了巨大的改变。如今，这座城市的俄罗斯历史遗产获得了高度重视并受到了精心保护。我在30年前看到的圣索菲亚大教堂几乎完全处于被遗弃和荒废的状态，而如今已完全修复，圣索菲亚大教堂甚至已经成为这座城市的标志性建筑之一。值得称赞的是，哈尔滨市政府甚至拆掉了教堂周围的建筑，重现了大教堂广场当初的盛况。\r\n\n\n\n\n\n哈尔滨索菲亚大教堂 图自IC photo\n\n\r\n\t还有哈尔滨火车站，1909年10月，朝鲜人安重根在此刺杀了日本首相伊藤博文，如今新落成的哈尔滨火车站新站房汲取了1899年沙俄时代老站房的一些元素，展现出了宏伟的欧洲新艺术风格。\r\n\n\r\n\t看到这些改变，我可以说：哈尔滨完全有理由把自己定位为中国与北方邻国进行经济、文化、教育和人

In [33]:
stream = client.chat.completions.create(
  model="qwen:14b",
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，简要说明一下。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

这些新闻摘要讲述了多方面的事件：

1. 张桂梅寄语高考学子，关注教育领域。

2. 有关苟仲文、唐一军被撤销全国政协委员资格的信息。

3. 盒马自有品牌出海的消息，反映了零售行业的全球化趋势。

4. 世界人工智能大会相关活动的发布，体现了科技领域的重大进展。

5. 其他涉及法律、安全和警示方面的信息。

## 直接用文本测试 LLM

In [104]:
%%time

file_path = './content.txt'

# Read the content of the file and store it in a string variable
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

CPU times: user 938 µs, sys: 0 ns, total: 938 µs
Wall time: 799 µs


In [105]:
%%time

stream = client.chat.completions.create(
  model="qwen:14b",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，详细归纳一下，形成要点列表。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

要点列表：

1. 欧洲的战略困境：冷战结束后的不稳定，军备竞赛、区域合作和多边机构的作用。

2. 亚洲的机会与挑战：地区战略关系的复杂性，大国竞争对安全的影响，以及域外大国的作用。

3. 全球安全倡议：中国提出的应对全球安全挑战的新方案，旨在推动构建人类命运共同体。

4. 结论：亚洲有结束冷战并向前迈进的战略机会。现在是通过合作迈向和平与繁荣的关键时期。CPU times: user 68 ms, sys: 5.95 ms, total: 73.9 ms
Wall time: 8.02 s


In [106]:
%%time

stream = client.chat.completions.create(
  model="qwen:14b",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，提炼要点并做适当的解释。要求基于原文意思解释。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

这篇文章讲述了欧洲在冷战后未能成功建立一个强大且全面的安全体系，而亚洲的情况更复杂，战略动荡持续，大国关系紧张。文章强调了保持大国间沟通的重要性，并指出俄中战略合作伙伴关系对亚洲和全球安全稳定起着基石作用。最后文章呼吁亚洲国家领导人以务实、负责任的方式共同努力，为建设“人类命运共同体”做出贡献。CPU times: user 51.4 ms, sys: 8.93 ms, total: 60.3 ms
Wall time: 7.19 s


In [92]:
len(content)

6290

In [94]:
%%time

file_path = './books/围城.txt'

# Read the content of the file and store it in a string variable
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 1.54 ms


In [95]:
len(content)

218562

In [103]:
%%time

stream = client.chat.completions.create(
  model="qwen:14b",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。"},
    {"role": "user", "content": f"文章内容：{content}\n\n主要讲了啥，提炼要点并做适当的解释。要求基于原文意思解释。"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

《围城》是钱钟书先生的代表作，是一部探讨婚姻与人生的长篇小说。

故事围绕主人公方鸿渐的生活展开。方鸿渐在国外留学期间，感情波折不断，最终娶了并不爱的女子孙柔嘉。婚后生活并不如意，两人的矛盾逐渐升级，导致家庭的不和谐。

《围城》通过主人公的生活经历和内心世界，揭示了婚姻、人际关系以及人生选择中的困境与无奈。这部作品语言幽默犀利，情节引人入胜，被誉为中国现代文学的经典之作。CPU times: user 78.3 ms, sys: 7.77 ms, total: 86 ms
Wall time: 8.28 s


In [102]:
%%time

stream = client.chat.completions.create(
  model="qwen:14b",
  max_tokens=5000,
  messages=[
    {"role": "system", "content": "你是一个概括内容和提炼主要内容的助手。要根据给出的文章内容回答。"},
    {"role": "user", "content": f"文章内容：{content}\n\n方鸿渐的老婆是谁"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

方鸿渐的老婆是孙柔嘉。CPU times: user 11.5 ms, sys: 4.29 ms, total: 15.8 ms
Wall time: 7.06 s


In [98]:
%%time

stream = client.chat.completions.create(
  model="qwen:14b",
  messages=[
    {"role": "system", "content": "你是一个回答各种问题的助手"},
    {"role": "user", "content": "方鸿渐的老婆是谁"}
  ],
  stream=True,
)

for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

方鸿渐的老婆是苏小姐，也就是后来改名为孙柔嘉的女性。在小说《围城》中，他们的婚姻生活也是故事的重要内容之一。CPU times: user 23.2 ms, sys: 5.72 ms, total: 28.9 ms
Wall time: 2.09 s


## 使用嵌入是否能直接从网页开始

直接加载文本做嵌入，效果是不行的。

In [40]:
%%time

# 导入需要的包

from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

CPU times: user 5.11 ms, sys: 0 ns, total: 5.11 ms
Wall time: 7.98 ms


In [41]:
%%time

# 初始化全局 embedding 模型
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="dztech/bge-large-zh:v1.5",
    # model_name="bge-m3:latest",
    base_url="http://192.168.0.72:11435",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.embed_model = ollama_embedding

CPU times: user 446 ms, sys: 45.5 ms, total: 491 ms
Wall time: 625 ms


In [42]:
%%time

url="https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml"
documents=get_documents(url)

CPU times: user 75.2 ms, sys: 1.9 ms, total: 77.1 ms
Wall time: 180 ms


In [43]:
%%time

# 生成faiss索引

import faiss

Settings.chunk_size=128
Settings.chunk_overlap=20

# dimensions of bge-large-zh
d = 1024
faiss_index = faiss.IndexFlatL2(d)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 903 ms, sys: 71.2 ms, total: 974 ms
Wall time: 17.8 s


In [44]:
%%time

# set Logging to DEBUG for more detailed outputs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100
)

streaming_response = query_engine.query("文章主要讲了啥")
streaming_response.print_response_stream()
print()

由于新的上下文没有提供具体的文章内容，我不能直接概括文章主题。根据之前的回答策略，我会重复原来的答案。

"对不起，由于提供的链接格式不正确，我无法访问并分析文章的内容。请确保链接的正确性，并提供详细的链接或问题描述，我将尽力帮助您概括文章的主题。"

请注意，如果新的上下文中有文章的具体内容，你应该重新提供问题，以便我能给出更准确的答案。
CPU times: user 312 ms, sys: 35.4 ms, total: 347 ms
Wall time: 35.1 s


In [51]:
documents[0].text[:1000]

'[![](../images/mian-logo.png)](/)\n\n[首页](/?s=dhshouye)\n\n[风闻](http://user.guancha.cn/?s=dhfengwen)\n\n[财经](/economy?s=dhcaijing "财经")\n\n[国际](/internation?s=dhguoji "国际")\n\n[军事](/military-affairs?s=dhjunshi "军事")\n\n[公司](/chanjing?s=dhchanjing "公司")\n\n[科技](/gongye·keji?s=dhgongye·keji "科技")\n\n[汽车](/qiche?s=dhqiche "汽车")\n\n[视频](/video/gczvideo/list.html "视频")\n\n__\n\n[![](https://i.guancha.cn/vip-diamond.gif)观察员](javascript:void\\(0\\))\n\n__ APP下载\n\n![](https://i.guancha.cn/app-erweima.png?20190610)\n\n扫一扫\n\n下载观察者APP\n\n  * ### 安德烈·科尔图诺夫：无论经济发展还是和平建设，亚洲都可以给欧洲上一课\n\n    * [![安德烈·科尔图诺夫](https://i.guancha.cn/authors/andelie/20220524105234640.png)](../andelie)\n\n[安德烈·科尔图诺夫](../andelie)俄罗斯国际事务委员会学术委员会主任\n\n分享到：\n\n____\n\n2024-06-06 08:12:41 字号：[A-](javascript:void\\(0\\);) [A](javascript:void\\(0\\);)\n[A+](javascript:void\\(0\\);) 来源：观察者网\n\n最后更新: 2024-06-06 13:23:14\n\n【文/观察者网专栏作者 安德烈·科尔图诺夫，翻译/马力】\n\n上世纪90年代初，我第一次来到了哈尔滨，这座掩映在丁香树林中的城市给我留下的印象是，这座城市正致力于消除俄罗斯残余痕迹，这是欧洲殖民主义最后一点影响的象征

## 尝试用 MainContentExtractorReader 生成 markdown 格式

In [56]:
%%time
%%capture

%pip install MainContentExtractor

from llama_index.readers.web import MainContentExtractorReader

def get_markdown_document(url):
    return MainContentExtractorReader().load_data(
        [url]
    )

CPU times: user 11.3 ms, sys: 7.44 ms, total: 18.8 ms
Wall time: 3.21 s


In [66]:
%%time

url="https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml" # parse error
url="https://www.163.com/dy/article/J40MKVM60514R9P4.html" # http 403
url="http://military.china.com.cn/2024-06/06/content_117238318.shtml" # 可以解析
url="https://baijiahao.baidu.com/s?id=1801081972774741387" # 乱码
url="https://new.qq.com/rain/a/20240606A067CK00" # 只有标题
documents=get_markdown_document(url)

documents[0].text

CPU times: user 57.9 ms, sys: 3.04 ms, total: 61 ms
Wall time: 132 ms


'仅500余股收涨，149家跌停！A股新风向：市值越大的越“香”？\n\n'

## 使用 GNE 抽取正文内容

可能比较适合新闻网站。

[GeneralNewsExtractor/GeneralNewsExtractor](https://github.com/GeneralNewsExtractor/GeneralNewsExtractor)

In [72]:
%%time
%%capture

%pip install gne

CPU times: user 9 ms, sys: 8.7 ms, total: 17.7 ms
Wall time: 2.96 s


In [78]:
%%time

from gne import GeneralNewsExtractor
import requests

url = "https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml"
response = requests.get(url)
html=response.text

extractor = GeneralNewsExtractor()
result = extractor.extract(soup, noise_node_list=['//div[@class="comment-list"]'])

result

CPU times: user 110 ms, sys: 0 ns, total: 110 ms
Wall time: 180 ms


{'title': '安德烈·科尔图诺夫：为什么二战后亚洲不像欧洲那样热衷建立多边联盟？',
 'author': '岳冠东',
 'publish_time': '2024-06-06 08:12:41',
 'content': '【文/观察者网专栏作者 安德烈·科尔图诺夫，翻译/马力】\n上世纪90年代初，我第一次来到了哈尔滨，这座掩映在丁香树林中的城市给我留下的印象是，这座城市正致力于消除俄罗斯残余痕迹，这是欧洲殖民主义最后一点影响的象征。\n事实上，这座始建于1898年的城市最初是由中东铁路线上的一个车站逐渐发展而来的，在其建成后的头20年里，哈尔滨具有特殊的治外法权地位，而且实际上已经脱离了首都北京的管辖。长期以来，当地中国人对这段国家主权受到损害的历史有着痛苦的回忆，而且对这段历史非常敏感，对此我们俄罗斯人必须接受并给予理解。\n然而，在过去的30多年里，人们的态度似乎发生了巨大的改变。如今，这座城市的俄罗斯历史遗产获得了高度重视并受到了精心保护。我在30年前看到的圣索菲亚大教堂几乎完全处于被遗弃和荒废的状态，而如今已完全修复，圣索菲亚大教堂甚至已经成为这座城市的标志性建筑之一。值得称赞的是，哈尔滨市政府甚至拆掉了教堂周围的建筑，重现了大教堂广场当初的盛况。\n哈尔滨索菲亚大教堂 图自IC photo\n还有哈尔滨火车站，1909年10月，朝鲜人安重根在此刺杀了日本首相伊藤博文，如今新落成的哈尔滨火车站新站房汲取了1899年沙俄时代老站房的一些元素，展现出了宏伟的欧洲新艺术风格。\n看到这些改变，我可以说：哈尔滨完全有理由把自己定位为中国与北方邻国进行经济、文化、教育和人文交流的国家级枢纽城市。\n在亚洲纪念第二次世界大战\n而对于俄罗斯人来说，哈尔滨也在历史上牵连着悠久而复杂的中俄关系。\n在第二次世界大战期间，约一万两千名苏联官兵为将中国东北从日本占领下解放出来而牺牲，前些日子俄罗斯总统普京来哈尔滨时，还向纪念这些苏联官兵的纪念碑献花。这位俄罗斯领导人特别注意到了中国为纪念两国共同的军事荣耀和兄弟情谊所表现出来的重视态度。这种对历史妥帖而及时的纪念，不仅适用于中国人，同样也适用于俄罗斯人。\n不可否认，与许多其他国家一样，俄罗斯人通常认为第二次世界大战主要是一场“发生在欧洲的战争”。如果你问一个普通俄罗斯人，20世纪最致命和最具破坏性的冲突是何时开

In [79]:
%%time

def get_extract_data(url):
    response = requests.get(url)
    html=response.text
    extractor = GeneralNewsExtractor()
    result = extractor.extract(soup, noise_node_list=['//div[@class="comment-list"]'])

    return result

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 7.87 µs


In [83]:
%%time

from llama_index.core import VectorStoreIndex, StorageContext, Document

url = "https://www.guancha.cn/andelie/2024_06_06_737185_s.shtml"
data=get_extract_data(url)

documents=[Document(text=data['content'])]


Settings.chunk_size=128
Settings.chunk_overlap=20

# dimensions of bge-large-zh
d = 1024
faiss_index = faiss.IndexFlatL2(d)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 393 ms, sys: 8.18 ms, total: 401 ms
Wall time: 9.12 s


In [85]:
%%time

# set Logging to DEBUG for more detailed outputs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=100
)

streaming_response = query_engine.query("文章主要讲了啥, 详细总结，形成列表要点说明")
streaming_response.print_response_stream()
print()

文章主要探讨了以下几个关键点：

1. **全球安全倡议（GSI））在中国的角色**：中国倡导的GSI反映了对全球安全挑战的日益重视。

2. **亚洲冷战与欧洲冷战的比较分析**：文章强调亚洲冷战可能更具复杂性和破坏性，不同于欧洲冷战模式。

3. **战略不稳定对亚洲地区的影响评估**：长期的战略不确定性可能导致亚洲地区的地缘政治格局变得更为敏感和动荡。

4. **大国间沟通与合作的重要性**：文章提醒国际社会，尤其是在大国之间，保持有效的沟通和协作对于维护和平与稳定至关重要。

5. **亚洲冲突的风险及潜在后果**：文章指出尽管亚洲的冲突可能不会升级到全面战争，但局部紧张局势仍然存在，并可能导致不稳定局面。
CPU times: user 433 ms, sys: 36 ms, total: 469 ms
Wall time: 1min 6s


In [86]:
%%time

# set Logging to DEBUG for more detailed outputs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=500
)

streaming_response = query_engine.query("文章主要讲了啥, 详细总结，形成列表要点说明")
streaming_response.print_response_stream()
print()

文章主要讨论了以下几个关键点：

1. **中国提出的全球安全倡议（GSI））**：习近平主席在博鳌亚洲论坛上提出了这个旨在应对全球安全挑战的倡议。

2. **亚洲冷战与欧洲冷战的比较**：文章强调了亚洲冷战的独特性和复杂性，并将其与欧洲冷战进行了对比分析。

3. **亚洲内部的战略动荡和安全威胁**：文章指出亚洲内部存在军事冲突的风险，以及各国在战略上的紧张关系，这可能对地区稳定构成挑战。

4. **大国间沟通的重要性**：文章强调了中美、中印等大国之间保持开放和透明的沟通对于维护地区稳定和避免误解的重要性。

5. **全球安全倡议面临的挑战与前景**：文章总结了全球安全倡议在全球安全领域可能带来的变革，同时也探讨了实施这一倡议所面临的具体挑战，以及未来的发展方向。
CPU times: user 494 ms, sys: 12.3 ms, total: 507 ms
Wall time: 1min 17s


In [87]:
%%time

# set Logging to DEBUG for more detailed outputs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=1000
)

streaming_response = query_engine.query("文章主要讲了啥, 详细总结，形成列表要点说明")
streaming_response.print_response_stream()
print()

文章主要讨论了以下几个关键点：

1. **亚洲冷战的特点**：文章强调了亚洲冷战不同于欧洲冷战的独特性和复杂性。

2. **中国的全球安全倡议（GSI）**：中国提出这一倡议，旨在促进全球范围内的安全合作与对话。

3. **战略关系和冲突风险的分析**：文章探讨了在亚洲地区存在的军备竞赛、潜在军事冲突以及大国间的战略互动。

4. **沟通与信任的重要性**：文章强调了在全球安全问题上保持开放沟通、建立互信机制的重要性。

5. **全球安全合作的挑战与前景展望**：最后，文章分析了当前全球安全合作面临的挑战，并对未来可能的合作趋势进行了预测。
CPU times: user 412 ms, sys: 24.5 ms, total: 437 ms
Wall time: 1min 13s
